#### **IMPORTAR BIBLIOTECAS**

In [11]:
from pyspark.sql.functions import *
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 13, Finished, Available, Finished)

#### **LEITURA DOS DADOS**

- **Gender:** Gênero. 
- **Weight:** Peso em kgs. 
- **family_history:** Algum membro da família sofreu ou sofre de excesso de 
- peso? 
- **FAVC:** Você come alimentos altamente calóricos com frequência? 
- **CAEC:** Você come alguma coisa entre as refeições?
- **SCC:** Você monitora as calorias que ingere diariamente? 
- **FAF:** Com que frequência você pratica atividade física? 
- **CALC:** Com que frequência você bebe álcool? 
- **MTRANS:** Qual meio de transporte você costuma usar?
- **Obesity_level (coluna alvo):** Nível de obesidade 

In [13]:
df = spark.sql("SELECT* FROM obesity") \
          .select('Genero', 'Peso', 'Historico_Familiar', 'FAVC', 'CAEC', 'SCC', 'FAF', 'CALC', 'MTRANS', 'Nivel_Obesidade')

df = df.toPandas()

display(df)

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 477ddd24-d760-4a00-a0f8-12c99143316f)

In [14]:
cat_cols = ['Genero', 'Historico_Familiar', 'FAVC', 'CAEC', 'SCC', 'CALC', 'MTRANS']

preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first'), cat_cols)
], remainder='passthrough')

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 16, Finished, Available, Finished)

In [16]:
X = df.drop('Nivel_Obesidade', axis=1)
y = df['Nivel_Obesidade']

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 18, Finished, Available, Finished)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 19, Finished, Available, Finished)

In [18]:
pipeline = Pipeline(steps=[
    ('preprocessamento', preprocessor),
    ('modelo', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 20, Finished, Available, Finished)

Pipeline(steps=[('preprocessamento',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['Genero',
                                                   'Historico_Familiar', 'FAVC',
                                                   'CAEC', 'SCC', 'CALC',
                                                   'MTRANS'])])),
                ('modelo', RandomForestClassifier(random_state=42))])

In [19]:
y_pred = pipeline.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 21, Finished, Available, Finished)

Acurácia: 0.8676122931442081
                    precision    recall  f1-score   support

  Obesidade Grau I       0.85      0.87      0.86        70
 Obesidade Grau II       0.92      0.93      0.93        60
Obesidade Grau III       1.00      1.00      1.00        65
 Peso Insuficiente       0.85      0.93      0.88        54
       Peso Normal       0.80      0.78      0.79        58
 Sobrepeso Nível I       0.84      0.74      0.79        58
Sobrepeso Nível II       0.80      0.81      0.80        58

          accuracy                           0.87       423
         macro avg       0.87      0.87      0.86       423
      weighted avg       0.87      0.87      0.87       423



In [20]:
# Selecionar uma amostra de teste
amostra = X_test.iloc[[76]]
real = y_test.iloc[76]

# Fazer a predição
predito = pipeline.predict(amostra)

print(f"Valor real: {real}")
print(f"Predição:   {predito[0]}")

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 22, Finished, Available, Finished)

Valor real: Obesidade Grau I
Predição:   Obesidade Grau I


In [21]:
# Selecionar uma linha aleatória do X_test
amostra = X_test.sample(1, random_state=79)
real = y_test.loc[amostra.index[0]]

# Fazer a predição
predito = pipeline.predict(amostra)

print(f"Valor real: {real}")
print(f"Predição:   {predito[0]}")

StatementMeta(, 960e4707-5914-4e18-b502-dbdab2a3cc3d, 23, Finished, Available, Finished)

Valor real: Peso Insuficiente
Predição:   Peso Insuficiente
